<a href="https://colab.research.google.com/github/sheldor07/Essay-Evaluator-Research-Assistantship/blob/main/topic_classification_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import pipeline

# Read the CSV file into a DataFrame
df = pd.read_csv('/content/essayData.csv')

# Initialize the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")


In [ ]:
df.head()

,Essay,Section,ABSTRACT,terms
0,1,1,Four in five Singaporeans say that they can co...,humanities
1,1,1,While the terms ÃfalsehoodsÃ and Ãfake news...,humanities
2,1,2,The digital era is characterised by easy creat...,humanities
3,1,2,It is important to be critical of fake news du...,"sciences,humanities"
4,1,3,Fake news has played an enormous role during C...,humanities


In [ ]:

# Create an empty list to store the predictions
predictions = []
df_random = df.sample(n=10)
# Iterate through each row of the DataFrame
for index, row in df_random.iterrows():
    # Get the value from the "Section" column
    sequence_to_classify = row['ABSTRACT']
    # Define the candidate labels
    candidate_labels = ['technology', 'business', 'humanities', 'sciences']

    # Perform zero-shot classification and get the predicted label
    prediction = classifier(sequence_to_classify, candidate_labels)

    # Add the predicted label to the list
    predictions.append(prediction['scores'])
    print(f'row :{row}/10')

# Add the predictions as a new column in the DataFrame
df_random['Predicted_Label'] = predictions

# Save the updated DataFrame to a new CSV file
df_random.to_csv('predicted_file.csv', index=False)


row :Essay                                                              4
Section                                                            7
ABSTRACT           Secondly, Singapore lacks the manpower to incr...
terms                                                       business
Predicted_Label    [0.4293269217014313, 0.34361210465431213, 0.16...
Word_Labels                                   [technology, business]
Name: 8, dtype: object/10
row :Essay                                                              4
Section                                                            7
ABSTRACT           Therefore, Singapore is looking to increase it...
terms                                                     humanities
Predicted_Label    [0.42720314860343933, 0.26210132241249084, 0.2...
Word_Labels                       [technology, business, humanities]
Name: 3, dtype: object/10
row :Essay                                                              6
Section                             

KeyboardInterrupt: ignored

In [ ]:
import pandas as pd
import ast

# Read the CSV file into a DataFrame
df = pd.read_csv('/content/predicted_file.csv')

# Dictionary mapping indices to labels
labels_dict = {
    0: 'technology',
    1: 'business',
    2: 'humanities',
    3: 'sciences'
}

# Create an empty list to store the word labels
word_labels = []

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    labels = ast.literal_eval(row['Predicted_Label'])
    word_label = []
    for idx, label in enumerate(labels):
        if float(label) > 0.2:  # Convert label to float before comparison
            word_label.append(labels_dict[idx])
    word_labels.append(word_label)

# Add the word labels as a new column in the DataFrame
df['Word_Labels'] = word_labels

# Save the updated DataFrame to a new CSV file
df.to_csv('predicted_file.csv', index=False)
